Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [90]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import math

## The *Nim* and *Nimply* classes

In [91]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [92]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [93]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [94]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [95]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [96]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [97]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [98]:
NUM_MATCHES = 100
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, gabriele)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [99]:
evaluate(make_strategy({"p": 0.9999}))

0.04

## P2 -> Evolutionary Alg 

In [100]:
def make_evolutionary_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if random.random() < genome["p"]:
            #PRENDI TUTTO
            rowChoice=random.choice([x[0] for x in enumerate(state.rows) if x[1]!=0])
            #print(rowChoice)
            ply=Nimply(rowChoice,state.rows[rowChoice])
        else:
            #PRENDI TUTTO -1
            rowChoice=random.choice([x[0] for x in enumerate(state.rows) if x[1]!=0])
            #print(rowChoice)
            ply=Nimply(rowChoice,state.rows[rowChoice] if state.rows[rowChoice]==1 else state.rows[rowChoice]-1)
        return ply

    return evolvable

In [101]:
NUM_GEN=100


def evolve(strat:Callable):
    genome={"p":0.5}
    prevWR=evaluate(strat(genome))
    direction=+0.05
    for i in range(NUM_GEN):
        genome["p"]+=direction
        newStrat=strat(genome)
        wr=evaluate(newStrat)
        if(prevWR<wr):
            direction*=1.25
        else :
            direction=-0.05*(direction/abs(direction)) #reset to 0.05 with flipped sign
        prevWR=wr
        print(f"Round {i+1}: WR:{wr} with P:{genome['p']}")
            
evolve(make_evolutionary_strategy)

Round 1: WR:0.47 with P:0.55
Round 2: WR:0.42 with P:0.6125
Round 3: WR:0.41 with P:0.5625
Round 4: WR:0.52 with P:0.6125
Round 5: WR:0.42 with P:0.675
Round 6: WR:0.49 with P:0.625
Round 7: WR:0.51 with P:0.5625
Round 8: WR:0.5 with P:0.484375
Round 9: WR:0.48 with P:0.534375
Round 10: WR:0.38 with P:0.48437500000000006
Round 11: WR:0.42 with P:0.534375
Round 12: WR:0.4 with P:0.596875
Round 13: WR:0.43 with P:0.546875
Round 14: WR:0.36 with P:0.484375
Round 15: WR:0.47 with P:0.534375
Round 16: WR:0.35 with P:0.596875
Round 17: WR:0.5 with P:0.546875
Round 18: WR:0.42 with P:0.484375
Round 19: WR:0.45 with P:0.534375
Round 20: WR:0.42 with P:0.596875
Round 21: WR:0.42 with P:0.546875
Round 22: WR:0.58 with P:0.596875
Round 23: WR:0.54 with P:0.659375
Round 24: WR:0.53 with P:0.609375
Round 25: WR:0.48 with P:0.659375
Round 26: WR:0.43 with P:0.609375
Round 27: WR:0.43 with P:0.659375
Round 28: WR:0.53 with P:0.609375
Round 29: WR:0.41 with P:0.546875
Round 30: WR:0.45 with P:0.596875

KeyboardInterrupt: 

## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (make_strategy({"p": 0.1}), optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 4>
DEBUG:root:status: After player 1 -> <1 3 3 7 9 11 13 15 17 19 4>
DEBUG:root:status: After player 0 -> <1 3 3 7 9 11 13 15 17 2 4>
DEBUG:root:status: After player 1 -> <1 3 3 7 9 11 13 15 0 2 4>
DEBUG:root:status: After player 0 -> <1 3 3 7 9 11 13 2 0 2 4>
DEBUG:root:status: After player 1 -> <1 3 3 7 4 11 13 2 0 2 4>
DEBUG:root:status: After player 0 -> <1 3 3 7 4 11 8 2 0 2 4>
DEBUG:root:status: After player 1 -> <1 3 3 2 4 11 8 2 0 2 4>
DEBUG:root:status: After player 0 -> <0 3 3 2 4 11 8 2 0 2 4>
DEBUG:root:status: After player 1 -> <0 2 3 2 4 11 8 2 0 2 4>
DEBUG:root:status: After player 0 -> <0 2 3 2 4 3 8 2 0 2 4>
DEBUG:root:status: After player 1 -> <0 2 3 2 4 3 0 2 0 2 4>
DEBUG:root:status: After player 0 -> <0 2 3 2 2 3 0 2 0 2 4>
DEBUG:root:status: After player 1 -> <0 2 3 2 2 3 0 2 0 2 2>
DEBUG:root:status: After player 0 -> <0 2 1 2 2 3 0 2 